In [2]:
%pip install pyzx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pyzx as zx

In [29]:
circuit = zx.Circuit(2)
circuit.add_gate("CNOT", 0, 1)

In [30]:
zx.draw(circuit)

In [32]:
circuit.to_graph().to_json()

'{"version": 2, "backend": "simple", "variable_types": {}, "scalar": {"power2": 1, "phase": "0"}, "inputs": [0, 1], "outputs": [4, 5], "vertices": [{"id": 0, "t": 0, "pos": [0, 0]}, {"id": 1, "t": 0, "pos": [0, 1]}, {"id": 2, "t": 2, "pos": [1, 1]}, {"id": 3, "t": 1, "pos": [1, 0]}, {"id": 4, "t": 0, "pos": [2, 0]}, {"id": 5, "t": 0, "pos": [2, 1]}], "edges": [[0, 3, 1], [1, 2, 1], [2, 3, 1], [2, 5, 1], [3, 4, 1]]}'

In [1]:
import numpy as np

class Cube:
    def __init__(self, position, color, rotation=0):
        self.position = np.array(position)  # [x, y, z]
        self.color = color  # 'green' or 'red'
        self.rotation = rotation  # Degrees around x-axis
        self.pipes = []  # List of (pipe, direction) tuples

    def attach_pipe(self, pipe, direction):
        self.pipes.append((pipe, direction))

class Pipe:
    def __init__(self, start_cube, end_cube, direction):
        self.start_cube = start_cube
        self.end_cube = end_cube
        self.direction = direction  # e.g., '+x', '-x', '+z', '-z'

def rotate_cube(cube, degrees):
    """Rotate the cube and its attached pipes around the x-axis."""
    cube.rotation += degrees
    cube.rotation %= 360

def connect_cubes(cube1, cube2, direction):
    """Connect two cubes with a pipe."""
    pipe = Pipe(cube1, cube2, direction)
    cube1.attach_pipe(pipe, direction)
    opposite_dir = '-x' if direction == '+x' else '+x'  # Simplified for x-axis
    cube2.attach_pipe(pipe, opposite_dir)

def transform_strict_heuristic(zx_graph):
    """Transform a PyZX graph into a 3D BlockGraph following your heuristic."""
    block_graph = []
    prev_color = None

    for row_index, row in enumerate(zx_graph):
        row_cubes = []
        for node_index, node in enumerate(row):
            position = [node_index, 0, row_index]
            cube = Cube(position, node['color'])
            row_cubes.append(cube)

            # Attach input pipe for the first node
            if node_index == 0:
                input_pipe = Pipe(None, cube, '-x')
                cube.attach_pipe(input_pipe, '-x')

            # Connect to the previous cube in the row
            if node_index > 0:
                prev_cube = row_cubes[node_index - 1]
                # Attach pipe *before* rotating
                connect_cubes(prev_cube, cube, '+x')
                # Rotate if color changes
                if node['color'] != prev_color:
                    rotate_cube(cube, 90)

            prev_color = node['color']

            # Attach output pipe for the last node
            if node_index == len(row) - 1:
                output_pipe = Pipe(cube, None, '+x')
                cube.attach_pipe(output_pipe, '+x')

        block_graph.append(row_cubes)

        # Connect to the previous row
        if row_index > 0:
            prev_row = block_graph[row_index - 1]
            for i, cube in enumerate(row_cubes):
                target_cube = prev_row[min(i, len(prev_row) - 1)]
                if not align_check(cube, target_cube):
                    extend_connection_strict(cube, target_cube)

    return block_graph

def align_check(cube1, cube2):
    """Check if two cubes align for a direct connection."""
    return cube1.position[2] == cube2.position[2] + 1 and cube1.position[0] == cube2.position[0]

def extend_connection_strict(start_cube, target_cube):
    """Add extra cubes and pipes to connect misaligned cubes."""
    current_position = start_cube.position.copy()
    direction = np.sign(target_cube.position - current_position)
    while not np.array_equal(current_position, target_cube.position):
        if direction[0] != 0:
            current_position[0] += direction[0]
        elif direction[2] != 0:
            current_position[2] += direction[2]
        new_cube = Cube(current_position, start_cube.color)
        connect_cubes(start_cube, new_cube, get_direction_string(direction))
        start_cube = new_cube

def get_direction_string(direction):
    """Convert direction vector to string."""
    if direction[0] > 0:
        return '+x'
    elif direction[0] < 0:
        return '-x'
    elif direction[2] > 0:
        return '+z'
    elif direction[2] < 0:
        return '-z'

# Example usage
zx_graph = [
    [{'color': 'green'}, {'color': 'red'}, {'color': 'green'}],  # Row 0
    [{'color': 'red'}, {'color': 'green'}]                      # Row 1
]

block_graph = transform_strict_heuristic(zx_graph)

# Print the structure
for row_idx, row in enumerate(block_graph):
    print(f"Row {row_idx}:")
    for cube in row:
        pipe_dirs = [dir for _, dir in cube.pipes]
        print(f"  Cube at {cube.position.tolist()}, Color: {cube.color}, Rotation: {cube.rotation}, Pipes: {pipe_dirs}")

Row 0:
  Cube at [0, 0, 0], Color: green, Rotation: 0, Pipes: ['-x', '+x']
  Cube at [1, 0, 0], Color: red, Rotation: 90, Pipes: ['-x', '+x']
  Cube at [2, 0, 0], Color: green, Rotation: 90, Pipes: ['-x', '+x']
Row 1:
  Cube at [0, 0, 1], Color: red, Rotation: 0, Pipes: ['-x', '+x']
  Cube at [1, 0, 1], Color: green, Rotation: 90, Pipes: ['-x', '+x']


In [2]:
import numpy as np

class Cube:
    def __init__(self, position, color):
        self.position = np.array(position)  # [x, y, z]
        self.color = color  # 'green' or 'red'
        self.pipes = []  # List of (pipe, direction) tuples

    def attach_pipe(self, pipe, direction):
        self.pipes.append((pipe, direction))

class Pipe:
    def __init__(self, start_cube, end_cube, direction):
        self.start_cube = start_cube
        self.end_cube = end_cube
        self.direction = direction  # e.g., '+x', '-x', '+z', '-z'

def connect_cubes(cube1, cube2, direction):
    """Connect two cubes with a pipe."""
    pipe = Pipe(cube1, cube2, direction)
    cube1.attach_pipe(pipe, direction)
    opposite_dir = '-z' if direction == '+z' else '+z'  # For z-axis
    cube2.attach_pipe(pipe, opposite_dir)

def transform_improved(zx_graph):
    """Transform a PyZX graph into a 3D BlockGraph using improved suggestions."""
    block_graph = []
    for z, row in enumerate(zx_graph):
        row_cubes = []
        for x, node in enumerate(row):
            position = [x, 0, z]  # (x, 0, z) for simplicity
            cube = Cube(position, node['color'])
            row_cubes.append(cube)

            # Connect to the previous row
            if z > 0:
                prev_cube = block_graph[z - 1][min(x, len(block_graph[z - 1]) - 1)]
                if align_check_improved(cube, prev_cube):
                    connect_cubes(prev_cube, cube, '+z')
                else:
                    extend_connection_improved(prev_cube, cube)

        block_graph.append(row_cubes)

    return block_graph

def align_check_improved(cube1, cube2):
    """Check if two cubes align along z."""
    return cube1.position[0] == cube2.position[0] and cube1.position[1] == cube2.position[1] and cube1.position[2] == cube2.position[2] + 1

def extend_connection_improved(start_cube, target_cube):
    """Add extra cubes to connect misaligned cubes."""
    current_position = start_cube.position.copy()
    while current_position[0] != target_cube.position[0]:
        step = 1 if target_cube.position[0] > current_position[0] else -1
        current_position[0] += step
        new_cube = Cube(current_position, start_cube.color)
        connect_cubes(start_cube, new_cube, '+x' if step > 0 else '-x')
        start_cube = new_cube
    connect_cubes(start_cube, target_cube, '+z')

# Example usage
zx_graph = [
    [{'color': 'green'}, {'color': 'red'}, {'color': 'green'}],  # Row 0
    [{'color': 'red'}, {'color': 'green'}]                      # Row 1
]

block_graph = transform_improved(zx_graph)

# Print the structure
for row_idx, row in enumerate(block_graph):
    print(f"Row {row_idx}:")
    for cube in row:
        pipe_dirs = [dir for _, dir in cube.pipes]
        print(f"  Cube at {cube.position.tolist()}, Color: {cube.color}, Pipes: {pipe_dirs}")

Row 0:
  Cube at [0, 0, 0], Color: green, Pipes: ['+z']
  Cube at [1, 0, 0], Color: red, Pipes: ['+z']
  Cube at [2, 0, 0], Color: green, Pipes: []
Row 1:
  Cube at [0, 0, 1], Color: red, Pipes: ['-z']
  Cube at [1, 0, 1], Color: green, Pipes: ['-z']
